![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Create SQL managed instance - Azure Arc on an existing Azure Arc Data Controller
 
This notebook walks through the process of creating a <a href="https://docs.microsoft.com/azure/sql-database/sql-database-managed-instance">SQL managed instance - Azure Arc</a> on an existing Azure Arc Data Controller.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target Azure Arc Data Controller already created.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Check prerequisites**

In [ ]:
import sys,os,json,subprocess
def run_command():
    print("Executing: " + cmd)
    output = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True )
    if output.returncode != 0:
        print(f'Command: {cmd} failed \n')
        print(f'\t>>>Error output: {output.stderr.decode("utf-8")}\n')
        sys.exit(f'exit code: {output.returncode}\n')
    print(f'Successfully executed: {cmd}')
    print(f'\t>>>Output: {output.stdout.decode("utf-8")}\n')
    return output.stdout.decode("utf-8")
cmd = 'azdata --version'
out = run_command()


#### **Ensure SQL instance name, username and password exist**

In [ ]:
# Required Values
env_var =  "AZDATA_NB_VAR_CONTROLLER_ENDPOINT" in os.environ
if env_var:
    controller_endpoint = os.environ["AZDATA_NB_VAR_CONTROLLER_ENDPOINT"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_CONTROLLER_ENDPOINT was not defined. Exiting\n')

env_var =  "AZDATA_NB_VAR_CONTROLLER_USERNAME" in os.environ
if env_var:
    controller_username = os.environ["AZDATA_NB_VAR_CONTROLLER_USERNAME"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_CONTROLLER_USERNAME was not defined. Exiting\n')

env_var =  "AZDATA_NB_VAR_CONTROLLER_PASSWORD" in os.environ
if env_var:
    controller_password = os.environ["AZDATA_NB_VAR_CONTROLLER_PASSWORD"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_CONTROLLER_PASSWORD was not defined. Exiting\n')

env_var =  "AZDATA_NB_VAR_SQL_INSTANCE_NAME" in os.environ
if env_var:
    mssql_instance_name = os.environ["AZDATA_NB_VAR_SQL_INSTANCE_NAME"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_SQL_INSTANCE_NAME was not defined. Exiting\n')

env_var = "AZDATA_NB_VAR_SQL_USERNAME" in os.environ
if env_var:
    mssql_username = os.environ["AZDATA_NB_VAR_SQL_USERNAME"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_SQL_USERNAME was not defined. Exiting\n')

env_var = "AZDATA_NB_VAR_SQL_PASSWORD" in os.environ
if env_var:
    mssql_password = os.environ["AZDATA_NB_VAR_SQL_PASSWORD"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_SQL_PASSWORD was not defined. Exiting\n')

env_var = "AZDATA_NB_VAR_SQL_STORAGE_CLASS_DATA" in os.environ
if env_var:
    mssql_storage_class_data = os.environ["AZDATA_NB_VAR_SQL_STORAGE_CLASS_DATA"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_SQL_STORAGE_CLASS_DATA was not defined. Exiting\n')

env_var = "AZDATA_NB_VAR_SQL_STORAGE_CLASS_LOGS" in os.environ
if env_var:
    mssql_storage_class_logs = os.environ["AZDATA_NB_VAR_SQL_STORAGE_CLASS_LOGS"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_SQL_STORAGE_CLASS_LOGS was not defined. Exiting\n')   


#### **Get optional parameters for the SQL instance**

In [ ]:
sql_instance_cores_request = os.environ.get("AZDATA_NB_VAR_SQL_CORES_REQUEST")
sql_instance_cores_limit = os.environ.get("AZDATA_NB_VAR_SQL_CORES_LIMIT")
sql_instance_memory_request = os.environ.get("AZDATA_NB_VAR_SQL_MEMORY_REQUEST")
sql_instance_memory_limit = os.environ.get("AZDATA_NB_VAR_SQL_MEMORY_LIMIT")

### **Creating the SQL managed instance - Azure Arc instance**

In [ ]:
# Login to the data controller.
#
os.environ["AZDATA_PASSWORD"] = controller_password
cmd = f'azdata login -e {controller_endpoint} -u {controller_username}'
out=run_command()

In [ ]:
print (f'Creating the SQL managed instance - Azure Arc instance')

cores_request_option = f' -cr "{sql_instance_cores_request}"' if sql_instance_cores_request else ""
cores_limit_option = f' -cl "{sql_instance_cores_limit}"' if sql_instance_cores_limit else ""
memory_request_option = f' -mr "{sql_instance_memory_request}Gi"' if sql_instance_memory_request else ""
memory_limit_option = f' -ml "{sql_instance_memory_limit}Gi"' if sql_instance_memory_limit else ""

os.environ["AZDATA_USERNAME"] = mssql_username
os.environ["AZDATA_PASSWORD"] = mssql_password
cmd = f'azdata arc sql mi create -n {mssql_instance_name} -scd {mssql_storage_class_data} -scl {mssql_storage_class_logs}{cores_request_option}{cores_limit_option}{memory_request_option}{memory_limit_option}'
out=run_command()